In [120]:
import torch
import sys
import numpy as np
import pandas as pd
import scanpy as sc
import os
import re
from anndata import AnnData
import matplotlib.pyplot as plt
import math
import random
from collections import defaultdict
from tqdm import tqdm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import warnings
import pingouin as pg
warnings.filterwarnings('ignore')

In [151]:
def find_best(node,n,max_len,min_len):
    global R_V
    global R_A
    R_V=list(np.zeros(n))
    for i in range(len(R_V)):
        R_V[i]=100
    R_A=list(np.zeros(n))
    find_best_step(node,max_len,min_len)
    return R_V,R_A
def find_best_step(node,max_len,min_len):
    global R_V
    global R_A
    if (max_len==-1 or len(node.A.columns)<=max_len) and (min_len==-1 or len(node.A.columns)>=min_len):
        I=-1
        for i in range(len(R_V)):
            if i==len(R_V)-1 and node.value<R_V[i]:
                I=i
                break
            if node.value>R_V[i]:
                I=i-1
                break
        if I>=0:
            R_V[I]=node.value
            R_A[I]=node.A
    for c_node in node.childs:
        find_best_step(c_node,max_len,min_len)
def compare(A,U_A):
    for u_A in U_A:
        if len(A.columns)==len(u_A.columns):
            lab=0
            for each in u_A.columns:
                lab2=0
                for each2 in A.columns:
                    if all(u_A[each]==A[each2]):
                        lab2=1
                        break
                if lab2==0:
                    lab=1
                    break
            if lab==0:
                return -1
    U_A.append(A)
class Node:
    def __init__(self,A):
        self.father=None
        self.head_node=None
        self.depth=0
        self.A=A
        self.dropped=[]
        self.value=0
        self.fixed=-1
        self.childs=[]
        self.select=[]
    def add_child(self,A):
        if compare(A,self.head_node.used_A)==-1:
            return -1
        t=Node(A)
        self.childs.append(t)
        t.depth=self.depth+1
        t.father=self
        t.fixed=t.father.fixed
        t.head_node=self.head_node
        return 1
    def calc(self,D,punishment=-1):
        if punishment==-1:
            punishment=100
            punishment=int(len(self.head_node.A.columns)/2)
        while 1:
            lab=0
            model=LinearRegression(fit_intercept=False,positive=True)
            model.fit(self.A,D)
            D_pre=pd.DataFrame(model.predict(self.A))
            E=pd.DataFrame(model.coef_)
            S=E.sum()
            for i in range(len(S)):
                #if S[i]==0:
                if S[i]<S.sum()/(len(S)*100):
                    lab=1
                    self.dropped.append(self.A[self.A.columns[i]])
                    self.A=self.A.drop(columns=[self.A.columns[i]])
                    compare(self.A,self.head_node.used_A)
                    break
            if lab==1:
                continue
            if len(E.columns)==1:
                S1=0
            else:
                #S1=(abs(E.corr()).sum().sum()-len(E.columns))/(len(E.columns)*min(punishment-1,len(E.columns-1)))
                #S1=(abs(E.corr()).sum().sum()-len(E.columns))/(len(E.columns))
                S1=(abs(E.corr()).sum().sum()-len(E.columns))/(len(E.columns)*min(punishment-1,len(E.columns-1)))
            S2=1-r2_score(D,D_pre)
            self.value=S1+S2
            break
        P=E.corr()
        P2=E.pcorr()
        for i in range(len(P)):
            P[i][i]=0
            P2[i][i]=0
        for EACH in [P.stack()[lambda x:x>0.2].nlargest(10).index,abs(P[P<0])[lambda x:x>0.2].stack().nlargest(10).index,P2.stack()[lambda x:x>0.2].nlargest(10).index,abs(P2[P2<0])[lambda x:x>0.2].stack().nlargest(10).index]:
            for each in EACH:
                if each[0]<each[1]:
                    if each not in self.select:
                        self.select.append(each)
        return  self.value

In [122]:
def f(node,D,patience,punishment):
    count=0
    count_keep=0
    value_keep=100
    searching=[node]
    while 1:
        count+=1
        if len(searching)==0 or count-count_keep>patience:
            break
        MV=100
        c_node=0
        i_keep=0
        for i in range(len(searching)):
            if searching[i].value<MV:
                MV=searching[i].value
                c_node=searching[i]
                i_keep=i
        del searching[i_keep]
        for each in c_node.select:
            e0=c_node.A.columns[each[0]]
            e1=c_node.A.columns[each[1]]
            if (c_node.A[e0]<=c_node.A[e1]).all():
                if each[0]<c_node.fixed:
                    continue
                A_plus=c_node.A.drop(columns=[e0])
            elif (c_node.A[e0]>=c_node.A[e1]).all():
                if each[1]<c_node.fixed:
                    continue
                A_plus=c_node.A.drop(columns=[e1])
            else:
                append=c_node.A[e0]+c_node.A[e1]
                append[lambda x:x>1]=1
                lab1=0
                for col in c_node.A:
                    if (append==c_node.A[col]).all():
                        lab1=1
                        break
                if lab1==1:
                    continue
                A_plus=pd.concat([c_node.A,append],axis=1,ignore_index=True)
            c_node.add_child(A_plus)
        for child in c_node.childs:
            V=child.calc(D,punishment)
            searching.append(child)
            if V<value_keep:
                value_keep=V
                count_keep=count
                #print(count,V) 
                #print(child.A)
def F(Ds,As=[],punishments=[],patiences=[],max_lens=[],min_lens=[],fixeds=[]):
    nodes=[]
    for i in range(len(Ds)):
        D=Ds[i]
        if len(fixeds)==0:
            fixed=-1
        else:
            fixed=fixeds[i]
        if len(As)==0:
            A=pd.DataFrame(np.identity(len(D)))
        else:
            A=As[i]
            if fixed==1:
                fixed=len(A.columns)
            I=pd.DataFrame(np.identity(len(D)))
            for i1 in I.columns:
                lab=0
                for j in A.columns:
                    if all(I[i1]==A[j]):
                        lab=1
                if lab==0:
                    A=pd.concat([A,I[i1]],axis=1,ignore_index=True)
        if len(punishments)==0:
            punishment=-1
        else:
            punishment=punishments[i]
        if len(patiences)==0:
            patience=30
        else:
            patience=patiences[i]
        if len(max_lens)==0:
            max_len=-1
        else:
            max_len=max_lens[i]
        if len(min_lens)==0:
            min_len=-1
        else:
            min_len=min_lens[i]
        H_node=Node(A)
        H_node.head_node=H_node
        H_node.fixed=fixed
        H_node.used_A=[]
        H_node.calc(D,punishment)
        f(H_node,D,patience,punishment)
        R_V,R_A=find_best(H_node,1,max_len,min_len)
        for i in range(len(R_V)):
            print('score: ',R_V[i],'\ncomb:\n',R_A[i])
        nodes.append(H_node)
    return nodes

# Test on generated data

In [5]:
def generate(types):
    nums=[]
    for i in range(len(types.columns)):
        a=np.random.randint(50,151)
        b=np.random.uniform(0.5,1.5)
        T=pd.DataFrame(np.random.normal(a*b,b*math.sqrt(a),size=10000)).astype(int)
        T[T<0]=0
        nums.append(T)
    N=pd.concat(nums,axis=1).T
    N=N.reset_index(drop=True)
    data=types.dot(N)
    L=10
    E=[]
    for i in data:
        E.append(i)
    for i in range(len(E)//L):
        data[str(i)+'_']=0
        for j in range(L):
            data[str(i)+'_']+=data[E[L*i+j]]
    data=data.drop(columns=E)
    return data

In [73]:
O_A0=pd.DataFrame(
    [[1,1,0,0,1],
     [0,1,0,0,1],
     [0,0,0,0,1],
     [0,0,1,0,1],
     [0,0,0,0,1],
     [0,0,1,1,1],
     [0,0,0,0,1]])
O_A1=pd.DataFrame(
    [[1,0,0],
     [1,1,1],
     [1,0,1],
     [0,1,0],
     [0,1,1],
     [0,1,0],
     [0,0,1]])
O_A2=pd.DataFrame(
    [[1,0,0,0,0,0],
     [1,1,0,0,0,0],
     [0,1,1,0,0,0],
     [0,0,1,1,0,0],
     [0,0,0,1,1,0],
     [0,1,0,0,1,1],
     [0,0,0,0,0,1]])
O_As=[O_A0,O_A1,O_A2]
Ds0=[]
for O_A in O_As:
    Ds0.append(generate(O_A))

In [78]:
F(Ds0,[])

score:  0.06880691054547547 
comb:
      0    1    3    4    5
0  1.0  0.0  0.0  1.0  1.0
1  0.0  0.0  0.0  1.0  1.0
2  0.0  0.0  0.0  0.0  1.0
3  0.0  0.0  1.0  0.0  1.0
4  0.0  0.0  0.0  0.0  1.0
5  0.0  1.0  1.0  0.0  1.0
6  0.0  0.0  0.0  0.0  1.0
score:  0.034327983133792074 
comb:
      2    3    4
0  0.0  0.0  1.0
1  1.0  1.0  1.0
2  1.0  0.0  1.0
3  0.0  1.0  0.0
4  1.0  1.0  0.0
5  0.0  1.0  0.0
6  1.0  0.0  0.0
score:  0.11943387471262963 
comb:
      2    3    4    5    6    7
0  1.0  0.0  0.0  0.0  0.0  0.0
1  1.0  1.0  0.0  0.0  0.0  0.0
2  0.0  1.0  0.0  0.0  0.0  1.0
3  0.0  0.0  0.0  0.0  1.0  1.0
4  0.0  0.0  0.0  1.0  1.0  0.0
5  0.0  1.0  1.0  1.0  0.0  0.0
6  0.0  0.0  1.0  0.0  0.0  0.0
